# TOP

In [1]:
# %% IMPORTS
import logging
import traceback
from datetime import datetime, timedelta
from io import StringIO
from pathlib import Path
from urllib.parse import quote_plus
import re

import pandas as pd
import pytz
import requests
from pandas import DataFrame as Df
from tqdm import tqdm

from db_engines import mms_db as db

from typing import Iterable


In [2]:
# %% input config ===================>
# Set to True to print debugging messages to stdout
we_are_debugging = True
# Set to True to avoid sending actual http requests, useful for debugging
# default should be False
testing_url_format = False

fn = 'AdHoc-20221202-Dialer_Lead_Upload.xlsx'

src_dir = Path('/mnt/c/Users/ecapo/OneDrive - Primedia Network, Inc/Reports/AdHoc')
src_pth = src_dir / fn

list_id = 103

sh_nm = 'Worksheet'
columns = {
    'Contact Phone (phase2)': 'phone',
    'Phone Number In Ad': 'phone_alt',

    'Lead Id': 'lead_id',

    'Company In Ad': 'company',
    'Company (phase2)': 'company_alt',

    'Lead Source': 'lead_source',

    'Vertical': 'vertical',
    'Media Market': 'market',

    'Contact Name (phase2)': 'name'
}
# == input config ===================<


# hard code config ==========================>


In [3]:
# == HTTP REQUEST URL CONFIG ===========>
api_url = 'http://10.1.10.20/vicidial/non_agent_api.php'

# these args stay the same for each url
api_static_args: dict[str, str] = {
    'source': 'test',
    'user': '6666',
    'pass': 'RedLakeSky3501',
    'function': 'add_lead',
    'list_id': str(list_id),
    'custom_fields': 'Y',
    'duplicate_check': 'DUPLIST30DAY',
}
# fields to update for each loop
variable_args: list[str] = [
    'phone_number',
    'MMS_Lead_ID',
    'Company_Name',
    'Lead_Source',
    'Website',
    'Vertical',
    'Media_Market',
    'first_name'
]

# fields that need a double quote string rather than spaces repalaced with '+'
# 'Lead_Source' may need to be included here
dblquote_params: list[str] = ['Website']
quote_params: list[str] = [
    s for s in variable_args
    # exlude pre defined fields and dbl qoulte fields
    # static args
    if (not s in dblquote_params) & (s in variable_args)
]
# <== HTTP REQUEST URL CONFIG <=========<


In [4]:
# for url formatting
quote_safe = '='  # character(s) to exclude in url encoding
# function to pass to url builder for regular encoding
def quote_reg(s): return quote_plus(s, safe=quote_safe)
# ditto for website and emails as url args, straight string in double quotes
def quote_quote(s): return f'"{s}"'

In [5]:
# LOGGING SETUP
logger_top_name = 'NotReached_dialer'

ansilg = '\x1b[93m'
ansirst = '\x1b[0m'
fmt_lg_ts = r'%Y-%m-%d %H:%M:%S'
fmt_lg_prfx = '[%(asctime)s||%(name)s:%(module)s:%(funcName)s||%(levelname)s]'
fmt_lg_msg = ' >> %(message)s'

ansi_green = '\x1b[32m'
confirm_log_msg = \
    f"\n{ansi_green}\t{'[URL]': >6}{ansirst}" +\
    "\t{u}" +\
    f"\n{ansi_green}\t{'[Resp]': >6}{ansirst}" +\
    "\t{response}\n"


fmt_lg_strm = f"{ansilg}{fmt_lg_prfx}{ansirst}{fmt_lg_msg}"

fmtr_strm: logging.Formatter = logging.Formatter(fmt=fmt_lg_strm, datefmt=fmt_lg_ts)

hdlr_strm: logging.Handler = logging.StreamHandler()
output_lvl: int = logging.DEBUG

hdlr_strm.setFormatter(fmtr_strm)

logger = logging.getLogger(logger_top_name)

logger.addHandler(hdlr_strm)

logger.setLevel(output_lvl)


logger.info(f"Let's go!")
logger.debug(f"Debugging...")



[2022-12-02 14:24:07||NotReached_dialer:747847263:<module>||INFO] >> Let's go!
[2022-12-02 14:24:07||NotReached_dialer:747847263:<module>||DEBUG] >> Debugging...


In [6]:
#FUNCTIONS
def query_url(
    api_url: str,
    arg_dict: dict,
    quote_params: Iterable[str],
    not_quote_params: Iterable[str],
    quote_norm,
    ampersand: bool = True,
    qmark: bool = True
) -> str:

    q: str = '?' if qmark else ''
    amp: str = '&' if ampersand else ''

    # f_args_str: str = urlencode(args)
    f_args_str: str = amp.join([
        f"{k}={quote_norm(v)}" if k in quote_params
        else f"{k}={v}"
        for k, v in arg_dict.items()
    ])
    return f"{api_url}{q}{f_args_str}"

def url_builder(df_: Df, api_args_static: dict[str, str]) -> list[str]:
    """spits out list of full url string

    Args:
        df_ (Df): _description_
        api_args_static (dict[str, str]): _description_

    Returns:
        list[str]: _description_
    """
    urls: list[str] = []

    for r in df_.itertuples():

        # some args are quote_plus, some are inside double quotes
        # this also elims the need for urllib.parse.urlencode
        args = {
            # no url encoding
            'phone_number': str(r.phone),
            'MMS_Lead_ID': str(r.lead_id),


            # regular quote_plus, '=' ok
            'Company_Name': str(r.company),
            'Lead_Source': str(r.lead_source),
            'Vertical': str(r.vertical),
            'Media_Market': str(r.market),
            'first_name': str(r.name)

            # bare string within ""
            # removes '-' lines from comments
            # 'Website': str(r.website),
        }


        # combine with recurring args
        api_args = api_args_static | args

        urls.append(
            query_url(
                api_url=api_url,
                arg_dict=api_args,
                quote_params=quote_params,
                not_quote_params=dblquote_params,
                quote_norm=lambda s: quote_plus(s, safe=quote_safe)
            ))

    return urls




# MAIN

In [7]:
# EXTRACT DF

results: Df = (
    pd.ExcelFile(src_pth)
    .parse(
        sheet_name=sh_nm,
        usecols=[c for c in columns.keys()]
    )
    .convert_dtypes()
    .rename(columns=columns)
)

results.phone = results.phone.fillna(results.phone_alt)
results.company = results.company.fillna(results.company_alt)

results = results.drop(columns=['phone_alt', 'company_alt'])


In [8]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   lead_id      181 non-null    Int64 
 1   lead_source  181 non-null    string
 2   vertical     73 non-null     string
 3   market       73 non-null     string
 4   company      180 non-null    string
 5   name         179 non-null    string
 6   phone        181 non-null    Int64 
dtypes: Int64(2), string(5)
memory usage: 10.4 KB


In [9]:
logger.debug(f"debugging: {we_are_debugging}, testing_only urls: {testing_url_format}")

[2022-12-02 14:24:07||NotReached_dialer:4244496977:<module>||DEBUG] >> debugging: True, testing_only urls: False


In [10]:
url_list = url_builder(results, api_static_args)

if testing_url_format:
    logger.debug(f"***Testing URL construction only. No HTTP requests will be executed.***")

for u in url_list:
    response = requests.Response() if testing_url_format else requests.get(u)

    # format response text for logging messages, the html confirmation for 200s has a return in it
    resp_log = f"resp: ({response.status_code}) >> {response.text}".replace('\n', '-|')

    logger.info(
        confirm_log_msg.format(
            u=u,
            response=resp_log if not testing_url_format
            else 'TESTING, NO REQUEST SENT'
        ))


[2022-12-02 14:24:08||NotReached_dialer:2833255198:<module>||INFO] >> 
	 [URL]	http://10.1.10.20/vicidial/non_agent_api.php?source=test&user=6666&pass=RedLakeSky3501&function=add_lead&list_id=103&custom_fields=Y&duplicate_check=DUPLIST30DAY&phone_number=7026861425&MMS_Lead_ID=147112&Company_Name=Vegas+Bridal&Lead_Source=Phone+In&Vertical=Entertainment&Media_Market=Las+Vegas&first_name=Bre+Mebane
	[Resp]	resp: (200) >> SUCCESS: add_lead LEAD HAS BEEN ADDED - 7026861425|103|4497|-8|6666-|NOTICE: add_lead CUSTOM FIELDS VALUES ADDED - 7026861425|4497|103|1-|

[2022-12-02 14:24:08||NotReached_dialer:2833255198:<module>||INFO] >> 
	 [URL]	http://10.1.10.20/vicidial/non_agent_api.php?source=test&user=6666&pass=RedLakeSky3501&function=add_lead&list_id=103&custom_fields=Y&duplicate_check=DUPLIST30DAY&phone_number=8476734422&MMS_Lead_ID=147934&Company_Name=MedWitness%2C+Ltd.&Lead_Source=Google+Paid+Search&Vertical=%3CNA%3E&Media_Market=%3CNA%3E&first_name=Bruce+Livingston
	[Resp]	resp: (200) >> 